In [0]:
%pip install fastapi uvicorn pyspark requests apscheduler


Python interpreter will be restarted.
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840651 sha256=aff7e37240d8d48ecbd94e84be63946e1e427d783338a5592a55709d5478aeb4
  Stored in directory: /root/.cache/pip/wheels/2e/d2/18/6f4f20e8332359f7fffceb6828edcc80ef96f86744192a7bb9
Successfully built pyspark
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Not uninstalling typing-extensions at /databricks/python3/lib/python3.9/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-bc0fc2b1-ba0a-40f6-af87-a5e34a3cd6bc
    Can't uninstall 'typing-extensions'. No files were found to uninstall.
Python interpreter will be restarted.


#### Integração com automação de atualização a cada hora

In [0]:
from fastapi import FastAPI, HTTPException
import uvicorn
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, StringType
import requests
import os
from apscheduler.schedulers.background import BackgroundScheduler

app = FastAPI()

# Configurando Spark
spark = SparkSession.builder.appName("FastAPI Spark Parquet").getOrCreate()

# Definir o esquema dos artigos
schema = StructType([
    StructField("source", StructType([
        StructField("id", StringType(), True),
        StructField("name", StringType(), True)
    ]), True),
    StructField("author", StringType(), True),
    StructField("title", StringType(), True),
    StructField("url", StringType(), True),
    StructField("urlToImage", StringType(), True),
    StructField("publishedAt", StringType(), True),
    StructField("content", StringType(), True)
])

# Definindo a API Key e o URL
API_KEY = "325474a96fcb4288bb357897851f51ce"
API_URL = "https://newsapi.org/v2/everything"
PARQUET_PATH = "/dbfs/FileStore/dados.parquet"

# Inicializando o agendador
scheduler = BackgroundScheduler()

def fetch_data():
    """
    Função que faz a requisição à API e retorna os dados como DataFrame do Spark.
    """
    params = {
        'q': "genomics or saúde or pesquisa",  # Palavras-chave relevantes
        'language': 'pt',  # Língua do artigo
        'sortBy': 'publishedAt',  # Ordenar por data de publicação
        'apiKey': API_KEY
    }

    # Fazer a requisição
    response = requests.get(API_URL, params=params)

    if response.status_code != 200:
        raise HTTPException(status_code=response.status_code, detail="Erro ao acessar a API")

    data = response.json()
    artigos = data['articles']

    # Converter para DataFrame do Spark
    df = spark.createDataFrame(artigos, schema=schema)
    return df


def update_parquet(df_atual, parquet_path):
    """
    Função que verifica se o arquivo Parquet existe e realiza a atualização.
    """
    # Verifica se o arquivo Parquet existe
    try:
        if os.path.exists(parquet_path):
            # Se o arquivo existir, lê-lo
            df_arquivo = spark.read.parquet(parquet_path)
            print("Arquivo Parquet existente lido com sucesso.")
            
            # Encontrar novos dados que não estão no Parquet
            df_novos_dados = df_atual.join(df_arquivo, on=["title"], how="left_anti")
            
            if df_novos_dados.count() > 0:
                # Gravar apenas novos dados
                df_novos_dados.write.mode("append").parquet(parquet_path)
                return f"{df_novos_dados.count()} novos registros adicionados ao Parquet."
            else:
                return "Nenhum dado novo para atualizar."

        else:
            # Se o arquivo não existir, gravar df_atual como Parquet
            df_atual.write.mode("overwrite").parquet(parquet_path)
            return "Arquivo Parquet não existia. Criado novo arquivo."

    except Exception as e:
        return f"Ocorreu um erro: {str(e)}"


def extrair_dados():
    """
    Função que extrai dados da API e armazena em Parquet, se necessário.
    """
    try:
        # Extrair dados da API
        df_atual = fetch_data()

        # Atualizar o arquivo Parquet
        resultado = update_parquet(df_atual, PARQUET_PATH)

        return {"message": resultado}

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))


@app.on_event("startup")
def start_scheduler():
    # Agendar a tarefa para executar a extração de dados a cada hora
    scheduler.add_job(extrair_dados, 'interval', hours=1)
    scheduler.start()


@app.on_event("shutdown")
def shutdown_scheduler():
    scheduler.shutdown()


@app.get("/")
def root():
    return {"message": "API de Atualização de Dados com FastAPI e Spark"}


# Executando o servidor
if __name__ == "__main__":
    config = uvicorn.Config(app)
    server = uvicorn.Server(config)
    await server.serve()    


<command-936036379599824>:109: DeprecationWarning: 
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
  @app.on_event("startup")
<command-936036379599824>:116: DeprecationWarning: 
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
  @app.on_event("shutdown")
INFO:     Started server process [2264]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:50488 - "GET /extradir_dados HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:38946 - "GET /extrair_dados HTTP/1.1" 404 Not Found


#### Integração com automação por requisição

In [0]:
from fastapi import FastAPI, HTTPException
import uvicorn
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, StringType
import requests
import os

app = FastAPI()

# Configurando Spark
spark = SparkSession.builder.appName("FastAPI Spark Parquet").getOrCreate()

# Definir o esquema dos artigos
schema = StructType([
    StructField("source", StructType([
        StructField("id", StringType(), True),
        StructField("name", StringType(), True)
    ]), True),
    StructField("author", StringType(), True),
    StructField("title", StringType(), True),
    StructField("url", StringType(), True),
    StructField("urlToImage", StringType(), True),
    StructField("publishedAt", StringType(), True),
    StructField("content", StringType(), True)
])

# Definindo a API Key e o URL
API_KEY = "325474a96fcb4288bb357897851f51ce"
API_URL = "https://newsapi.org/v2/everything"
PARQUET_PATH = "/dbfs/FileStore/dados.parquet"

def fetch_data():
    """
    Função que faz a requisição à API e retorna os dados como DataFrame do Spark.
    """
    # Parâmetros de consulta
    params = {
        'q': "genomics OR genomic OR 'genetic sequencing' OR 'DNA research' OR 'genetic analysis'",  # Palavras-chave relevantes
        'language': 'pt',  # Língua do artigo
        'sortBy': 'publishedAt',  # Ordenar por data de publicação
        'apiKey': API_KEY
    }

    # Fazer a requisição
    response = requests.get(API_URL, params=params)
    
    if response.status_code != 200:
        raise HTTPException(status_code=response.status_code, detail="Erro ao acessar a API")

    data = response.json()
    artigos = data['articles']

    # Converter para DataFrame do Spark
    df = spark.createDataFrame(artigos, schema=schema)
    return df

def update_parquet(df_atual, parquet_path):
    """
    Função que verifica se o arquivo Parquet existe e realiza a atualização.
    """
    try:
        # Verifica se o arquivo Parquet existe usando dbutils
        files = dbutils.fs.ls(os.path.dirname(parquet_path))
        file_names = [file.name for file in files]

        if os.path.basename(parquet_path) in file_names:
            # Se o arquivo existir, lê-lo
            df_arquivo = spark.read.parquet(parquet_path)
            print("Arquivo Parquet existente lido com sucesso.")
            
            # Encontrar novos dados que não estão no Parquet
            df_novos_dados = df_atual.join(df_arquivo, on=["title"], how="left_anti")
            
            if df_novos_dados.count() > 0:
                # Gravar apenas novos dados
                df_novos_dados.write.mode("append").parquet(parquet_path)
                return f"{df_novos_dados.count()} novos registros adicionados ao Parquet."
            else:
                return "Nenhum dado novo para atualizar."

        else:
            # Se o arquivo não existir, gravar df_atual como Parquet
            df_atual.write.mode("overwrite").parquet(parquet_path)
            return "Arquivo Parquet não existia. Criado novo arquivo."

    except Exception as e:
        return f"Ocorreu um erro: {str(e)}"

@app.get("/extrair_dados")
def extrair_dados():
    """
    Endpoint para extrair dados da API e armazenar em Parquet, se necessário.
    """
    try:
        # Extrair dados da API
        df_atual = fetch_data()

        # Atualizar o arquivo Parquet
        resultado = update_parquet(df_atual, PARQUET_PATH)

        return {"message": resultado}

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/")
def root():
    return {"message": "API de Atualização de Dados com FastAPI e Spark"}

if __name__ == "__main__":
    config = uvicorn.Config(app)
    server = uvicorn.Server(config)
    await server.serve()    


java.net.NoRouteToHostException: No route to host
	at java.base/sun.nio.ch.SocketChannelImpl.checkConnect(Native Method)
	at java.base/sun.nio.ch.SocketChannelImpl.finishConnect(SocketChannelImpl.java:777)
	at shaded.v9_4.org.eclipse.jetty.io.SelectorManager.doFinishConnect(SelectorManager.java:355)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector.processConnect(ManagedSelector.java:347)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector.access$1700(ManagedSelector.java:65)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector$SelectorProducer.processSelected(ManagedSelector.java:676)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector$SelectorProducer.produce(ManagedSelector.java:535)
	at shaded.v9_4.org.eclipse.jetty.util.thread.strategy.EatWhatYouKill.produceTask(EatWhatYouKill.java:362)
	at shaded.v9_4.org.eclipse.jetty.util.thread.strategy.EatWhatYouKill.doProduce(EatWhatYouKill.java:186)
	at shaded.v9_4.org.eclipse.jetty.util.thread.strategy.EatWhatYouKill.tryProduce(EatWhatYouKill.java:173)
	at shaded.v9_4.org.eclipse.jetty.util.thread.strategy.EatWhatYouKill.run(EatWhatYouKill.java:131)
	at shaded.v9_4.org.eclipse.jetty.util.thread.ReservedThreadExecutor$ReservedThread.run(ReservedThreadExecutor.java:409)
	at com.databricks.rpc.ShadedInstrumentedQueuedThreadPool$$anon$2.$anonfun$run$4(InstrumentedQueuedThreadPool.scala:183)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.logging.AttributionContextTracing.$anonfun$withAttributionContext$1(AttributionContextTracing.scala:48)
	at com.databricks.logging.AttributionContext$.$anonfun$withValue$1(AttributionContext.scala:273)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.AttributionContext$.withValue(AttributionContext.scala:269)
	at com.databricks.logging.AttributionContextTracing.withAttributionContext(AttributionContextTracing.scala:46)
	at com.databricks.logging.AttributionContextTracing.withAttributionContext$(AttributionContextTracing.scala:43)
	at com.databricks.rpc.ShadedInstrumentedQueuedThreadPool.withAttributionContext(InstrumentedQueuedThreadPool.scala:130)
	at com.databricks.rpc.ShadedInstrumentedQueuedThreadPool$$anon$2.$anonfun$run$3(InstrumentedQueuedThreadPool.scala:183)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.instrumentation.QueuedThreadPoolInstrumenter.trackActiveThreads(QueuedThreadPoolInstrumenter.scala:126)
	at com.databricks.instrumentation.QueuedThreadPoolInstrumenter.trackActiveThreads$(QueuedThreadPoolInstrumenter.scala:123)
	at com.databricks.rpc.ShadedInstrumentedQueuedThreadPool.trackActiveThreads(InstrumentedQueuedThreadPool.scala:130)
	at com.databricks.rpc.ShadedInstrumentedQueuedThreadPool$$anon$2.run(InstrumentedQueuedThreadPool.scala:177)
	at shaded.v9_4.org.eclipse.jetty.util.thread.QueuedThreadPool.runJob(QueuedThreadPool.java:883)
	at shaded.v9_4.org.eclipse.jetty.util.thread.QueuedThreadPool$Runner.run(QueuedThreadPool.java:1034)
	at java.base/java.lang.Thread.run(Thread.java:829)